## Device check

In [1]:
!nvidia-smi

Fri Oct 20 03:15:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:15:00.0 Off |                    0 |
| N/A   27C    P0    42W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Packages

In [2]:
from matplotlib import pyplot
import numpy as np
import logging
logging.basicConfig(level="INFO")
import os

import pandas as pd
from pprint import pprint
from transformers import (AutoTokenizer, AutoModel, AdamW, AutoConfig, get_linear_schedule_with_warmup)
from torch.utils.data import DataLoader

import sys
sys.path.append("../")

/ocean/projects/cis230036p/cmanouan/miniconda3/envs/smf_be/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import config
from dataloader import BEDataset, BEDataModule

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp1382fjii
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp1382fjii/_remote_module_non_scriptable.py


In [6]:
csv = pd.read_csv(os.path.join(config.DATASET_PATH, "train.csv"))

csv.head(n=2)

,sample_ID,in_state,goal_state,action_description,motor_cmd,len_action_desc,len_motor_cmd,version
0,7294,0,10,put the fork to the right of buttermilk,:FORK GREEN POSE-9 :BUTTERMILK GREEN POSE-2 :F...,8,11,v2
1,405,0,8,move the bottle backwards,:BOTTLE RED POSE-2 :BOTTLE #'*backward-transf...,4,8,v1


In [7]:
# building data object
ds = BEDataset(
    df=csv    
)

len(ds)

4876

In [8]:
# fetching example
rand_idx = np.random.randint(low=0, high=len(ds))
ex = ds[rand_idx]

print("Dataset size: ", len(ds))
print("="*100)
print("ID\t: ", ex["sample_id"])
print(">> InState\t: ", ex["in_state"].shape)
print(">> Desc\t:")
pprint(ex["action_desc"])
print(">> Cmd\t:")
pprint(ex["motor_cmd"])
print("="*100)

Dataset size:  4876
ID	:  9401
>> InState	:  torch.Size([3, 224, 224])
>> Desc	:
{'ids': tensor([  101,  2404,  1996,  5442,  2000,  1996,  2157,  1997, 12256, 17130,
         2378,   102,     0,     0,     0]),
 'length': 8,
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]),
 'raw': 'put the knife to the right of mondamin',
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
>> Cmd	:
{'ids': tensor([ 0, 17, 41, 30, 18, 41, 36, 17, 48, 18,  2,  2,  2,  2,  2]),
 'length': 11,
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]),
 'raw': ':KNIFE GREEN POSE-10 :MONDAMIN GREEN POSE-3 :KNIFE  '
        "#'*rightward-transformation*  :MONDAMIN"}


In [9]:
ex["action_desc"]['raw']

'put the knife to the right of mondamin'

In [10]:
ex["action_desc"]['ids']

tensor([  101,  2404,  1996,  5442,  2000,  1996,  2157,  1997, 12256, 17130,
         2378,   102,     0,     0,     0])

In [11]:
ds._decode_inputs(ids=ex["action_desc"]['ids'].tolist())

'put the knife to the right of mondamin'

In [12]:
ex["motor_cmd"]['ids']

tensor([ 0, 17, 41, 30, 18, 41, 36, 17, 48, 18,  2,  2,  2,  2,  2])

In [13]:
ds._decode_outputs(ids=ex["motor_cmd"]['ids'].tolist())

":KNIFE GREEN POSE-10 :MONDAMIN GREEN POSE-3 :KNIFE #'*rightward-transformation* :MONDAMIN"

In [14]:
def denormalize_img(
    normalized_image, 
    mean:list=config.MEAN, 
    std:list=config.STD
):
    denormalized_image = normalized_image.clone()
    for i in range(3):
        denormalized_image[i] = (denormalized_image[i] * std[i]) + mean[i]
    
    denormalized_image = denormalized_image.permute(1, 2, 0).numpy()
    
    return denormalized_image


In [15]:
# ## PS: we only de-normalize images for better visualization
# for _ in range(5):
#     rand_idx = np.random.randint(low=0, high=len(ds))
#     ex = ds[rand_idx]

#     fig, ax = pyplot.subplots(nrows=1, ncols=2, figsize=(8, 4))
#     in_ = denormalize_img(ex["in_state"])
#     goal_ = denormalize_img(ex["goal_state"])

#     ax[0].imshow(in_)
#     ax[1].imshow(goal_)

#     desc = ex["action_desc"]["raw"]
#     fig.suptitle(f"Instruction: {desc}")

#     pyplot.tight_layout()
#     pyplot.show()

#     # break

## Data loading pipeline

### Create data module

In [16]:
dm = BEDataModule()
dm.setup()

Total # examples: 4876


INFO:root:Training on 3610 samples.
INFO:root:Validating on 1266 samples.


### Test data loading I/O

In [18]:
print("="*100)
logging.info("\n>> train data loader")
print(f"# train batches\t: {len(dm.train_dataloader())}")
for data in dm.train_dataloader():
    # pprint(data)
    sample_id, in_state, ad, cmd = data["sample_id"], data["in_state"], data["action_desc"], data["motor_cmd"]
    print("In \t\t\t: ", in_state.shape)
    print("Action desc \t\t: ", ad["ids"].shape)
    print("Action desc (len) \t: ", ad["length"].shape)
    print("CMD \t\t\t: ", cmd["ids"].shape)
    print("CMD(len) \t\t: ", cmd["length"].shape)
    break

logging.info("\n\n>> val data loader")
print(f"# validation batches\t: {len(dm.val_dataloader())}")
for data in dm.val_dataloader():
    # pprint(data)
    sample_id, in_state, ad, cmd = data["sample_id"], data["in_state"], data["action_desc"], data["motor_cmd"]
    print("In \t\t\t: ", in_state.shape)
    print("Action desc \t\t: ", ad["ids"].shape)
    print("Action desc (len) \t: ", ad["length"].shape)
    print("CMD \t\t\t: ", cmd["ids"].shape)
    print("CMD(len) \t\t: ", cmd["length"].shape)
    break
    
print("="*100)

INFO:root:
>> train data loader


# train batches	: 452
In 			:  torch.Size([8, 3, 224, 224])
Action desc 		:  torch.Size([8, 15])
Action desc (len) 	:  torch.Size([8])
CMD 			:  torch.Size([8, 15])
CMD(len) 		:  torch.Size([8])


INFO:root:

>> val data loader


# validation batches	: 159
In 			:  torch.Size([8, 3, 224, 224])
Action desc 		:  torch.Size([8, 15])
Action desc (len) 	:  torch.Size([8])
CMD 			:  torch.Size([8, 15])
CMD(len) 		:  torch.Size([8])
